# 0. Импорт библиотек, задание функций

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split

In [21]:
# функция косинусной близости
def cosine_similarity(vec1, vec2): 
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (norm_vec1 * norm_vec2)

# 1. Загрузка датасета, разбиение на train test, создание матрицы

In [5]:
ratings = pd.read_csv('movie_data/ratings.csv')
movies = pd.read_csv('movie_data/movies.csv')

print('Размер датасета:', ratings.shape)
print('Уникальные пользователи:', ratings['userId'].nunique())
print('Уникальные фильмы:', ratings['movieId'].nunique())

Размер датасета: (100836, 4)
Уникальные пользователи: 610
Уникальные фильмы: 9724


In [24]:
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

In [31]:
unique_users = np.unique(ratings.to_numpy()[:, 0]) # уникальные юзеры
unique_movies = np.unique(ratings.to_numpy()[:, 1]) # уникальные фильмы

user_id_dict = {user_id: i for i, user_id in enumerate(unique_users)}
movie_id_dict = {movie_id: i for i, movie_id in enumerate(unique_movies)}

In [35]:
train_data

,userId,movieId,rating,timestamp
80568,509,7347,3.0,1435994597
50582,326,71462,4.0,1322252335
8344,57,2115,3.0,965798155
99603,610,1127,4.0,1479544102
71701,462,2409,2.0,1174438249
...,...,...,...,...
6265,42,4005,4.0,996259059
54886,364,141,4.0,869443367
76820,480,6867,4.0,1179163171
860,6,981,3.0,845556567


# 2. User-based

In [37]:
user_item_matrix = np.zeros((len(unique_users), len(unique_movies)))
for user_id, movie_id, rating, _ in train_data.to_numpy():
    user_idx = user_id_dict[user_id]
    movie_idx = movie_id_dict[movie_id]
    user_item_matrix[user_idx, movie_idx] = rating

In [39]:
def predict_user_based(user_id, movie_id):
    if user_id not in user_id_dict or movie_id not in movie_id_dict:
        return np.nan
    u_idx = user_id_dict[user_id]
    i_idx = movie_id_dict[movie_id]
    user_ratings = user_item_matrix[u_idx,:] # ср рейтинг пользователя
    r_u_bar = user_ratings[user_ratings > 0].mean() if np.any(user_ratings > 0) else 0
    chisl = 0
    znam = 0
    for v_idx in range(len(unique_users)):
        if v_idx != u_idx and user_item_matrix[v_idx, i_idx] > 0:
            sim_uv = cosine_similarity(user_item_matrix[u_idx, :], user_item_matrix[v_idx, :])
            r_v_bar = user_item_matrix[v_idx, :][user_item_matrix[v_idx, :] > 0].mean() if np.any(user_item_matrix[v_idx, :] > 0) else 0
            
            chisl += sim_uv * (user_item_matrix[v_idx, i_idx] - r_v_bar)
            znam += abs(sim_uv)
    if znam == 0:
        return r_u_bar
    
    return r_u_bar + (chisl / znam)

In [41]:
predicted_ratings_user = []
true_ratings_user = []

for user_id, movie_id, rating, _ in test_data.to_numpy():
    predicted = predict_user_based(user_id, movie_id)
    if not np.isnan(predicted):
        predicted_ratings_user.append(predicted)
        true_ratings_user.append(rating)

rmse_user_based = np.sqrt(np.mean((np.array(true_ratings_user) - np.array(predicted_ratings_user))**2))

print(f'RMSE для User-Based рекомендательной системы: {rmse_user_based:.4f}')

RMSE для User-Based рекомендательной системы: 0.8992


# 3. Item-based

In [58]:
item_user_matrix = np.zeros((len(unique_movies), len(unique_users)))
for user_id, movie_id, rating, _ in train_data.to_numpy():
    user_idx = user_id_dict[user_id]
    movie_idx = movie_id_dict[movie_id]
    item_user_matrix[movie_idx, user_idx] = rating

In [60]:
def predict_item_based(user_id, movie_id):
    if user_id not in user_id_dict or movie_id not in movie_id_dict:
        return np.nan
    u_idx = user_id_dict[user_id]
    i_idx = movie_id_dict[movie_id]
    item_ratings = item_user_matrix[i_idx, :]
    r_i_bar = item_ratings[item_ratings > 0].mean() if np.any(item_ratings > 0) else 0
    chisl = 0
    znam = 0
    for j_idx in range(len(unique_movies)):
        if j_idx != i_idx and item_user_matrix[j_idx, u_idx] > 0:
            sim_ij = cosine_similarity(item_user_matrix[i_idx, :], item_user_matrix[j_idx, :])
            r_j_bar = item_user_matrix[j_idx, :][item_user_matrix[j_idx, :] > 0].mean() if np.any(item_user_matrix[j_idx, :] > 0) else 0
            
            chisl += sim_ij * (item_user_matrix[j_idx, u_idx] - r_j_bar)
            znam += abs(sim_ij)
    if znam == 0:
        return r_i_bar

    return r_i_bar + (chisl / znam)

In [62]:
predicted_ratings_item = []
true_ratings_item = []

for user_id, movie_id, rating, _ in test_data.to_numpy():
    predicted = predict_item_based(user_id, movie_id)
    if not np.isnan(predicted):
        predicted_ratings_item.append(predicted)
        true_ratings_item.append(rating)

rmse_item_based = np.sqrt(np.mean((np.array(true_ratings_item) - np.array(predicted_ratings_item))**2))
print(f"RMSE для Item-Based рекомендательной системы: {rmse_item_based:.4f}")

RMSE для Item-Based рекомендательной системы: 1.1044


# 4. Сравнение подходов

На киношных данных User-Based подход оказался более точным, так как его  RMSE (0.8992) ниже, чем у Item-Based подхода.

Это говорит о том, что предсказания, основанные на схожести пользователей, лучше соответствуют реальным оценкам.


При этом item-based подход считался гораздо дольше, т.к. фильмов в датасете больше.


**Общие выводы по подходам:**
- User-Based подход может давать более точные рекомендации, он менее масштабируем и эффективен в условиях, когда количество пользователей очень велико. 
- Item-Based подход более практичен для крупных систем, так как вычисления схожести между фильмами обычно происходят реже, чем между пользователями, что делает его более подходящим для "холодного старта" новых пользователей.